<a href="https://colab.research.google.com/github/saadoonhammad/Urban_Noise_Anomaly_TinyML/blob/main/189_LSTM_Anomalies_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip show tensorflow

In [ ]:
pip install -U kaleido

In [ ]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_KERAS= MODELS_DIR + 'keras.h5'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
# MODEL_DYN_TFLITE = MODELS_DIR + 'model_dyn_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [ ]:
#Importing Libaries
import os
import random
from os import listdir
from io import StringIO
from os.path import join
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly.express as px
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import median_abs_deviation
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler, MaxAbsScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

In [ ]:
#IMPORTING DATA
helsinki=pd.read_csv('/path/LAeq-2019-all.csv')
helsinki.time= pd.to_datetime(helsinki['time'], unit='ms').astype('datetime64')
helsinki['date']=helsinki['time'].dt.date.astype('datetime64')
helsinki['hour']=helsinki['time'].dt.hour
helsinki['dow']= helsinki['time'].dt.day_name()
helsinki['month']=helsinki['time'].dt.month
helsinki['day']=helsinki['time'].dt.day
helsinki['nod']=pd.Categorical(helsinki['dow'], categories=
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'],
    ordered=True)
df_helsinki= helsinki [['time', 'dBA', 'dev-id', 'date', 'hour', 'dow', 'month', 'day', 'nod']]
df_helsinki.head()

In [ ]:
 #DICTIONARY FOR KEY-VALUE PAIRS IDENTIFYING EACH SENSOR ID WITH A UNIQUE NUMBER
 sensors = {"TA120-T246177" : 2, "TA120-T246182" : 3,"TA120-T246183" : 4,"TA120-T246184" : 5,
            "TA120-T246187" : 6,"TA120-T246189" : 7,"TA120-T246191" : 8}

In [ ]:
#MAPPING THE KEYS 'dev-id' IN DATAFRAME WITH VALUES 
df_helsinki['sensor'] = df_helsinki['dev-id'].map(sensors)

In [ ]:
#SELECTING THE SENSOR WITH ID TA120-T246189
df_189= df_helsinki.loc[(df_helsinki ['sensor'] == 7)  ]
df_189.count()

In [ ]:
fig= px.line(df_189, x='time', y='dBA',  title='Scatterplot of sound values')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
#REMOVING ANY VALUES FROM YEAR 2019
df_189= df_189[df_189['date'] > '2018-12-31']
df_189.shape

In [ ]:
#DUPLICATES REMOVAL BASED ON TIME COLUMN. 
df_189 = df_189.drop_duplicates(subset = ["time"], keep='first')
df_189.count()

In [ ]:
df_189.groupby('date').filter(lambda g: len(g) < 1439).groupby('date').size().sort_values(ascending=False)

In [ ]:
#rds
df_189_new= pd.DataFrame(df_189.groupby('date').filter(lambda g: len(g) < 1439).groupby('date').size().sort_values(ascending=False))
df_189_new.index

In [ ]:
df_189= df_189.drop(df_189.loc[df_189['date'].isin(df_189_new.index)].index)

In [ ]:
#CHECK FOR NULL VALUES
df_189.isnull().values.any()

In [ ]:
#SORT VALUES BY TIME
df_189=df_189.sort_values('time')

In [ ]:
#DATASTATS AFTER CLEANING OF DATA
df_189.describe()

In [ ]:
fig= px.line(df_189, x='time', y='dBA',  title='Lineplot of sound values')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
df_189_hourly=df_189.groupby(['hour'])['hour'].count()
fig= px.bar(df_189_hourly, text='value', labels={ 'index': 'hour', 'value':'count'}, title='Total count per hour', width=1000,height=600)
fig.show()

In [ ]:
df_189_hourly_mean=df_189[['dBA', 'hour']].groupby(['hour']).mean()
fig= px.line(df_189_hourly_mean,text=df_189_hourly_mean.index, title='Sound values aggregated by hour',
             labels={'value':'Noise Value (dBA)', 'hour': 'Hour'}, width=1000,height=600)
fig.update_traces(textposition='top right')
fig.show()

In [ ]:
daily_count=pd.DataFrame((df_189.groupby(['date'])['dBA'].count()))
daily_count
daily_count.loc[(daily_count['dBA']< 1440) & (daily_count.index > '2019-01-01')]

In [ ]:
df_189_daily=df_189.groupby(['date'])['date'].count()
fig= px.bar(df_189_daily, text='value', labels={ 'index': 'Day', 'value':'count'}, title='Total count per day', width=1500,height=600)
fig.show()

In [ ]:
df_189_daily_mean=df_189[['dBA', 'date']].groupby(['date']).mean()
fig= px.line(df_189_daily_mean, title='Sound values aggregated by day',
             labels={'value':'Noise Value (dBA)', 'date': 'Date'}, width=1000,height=700)
# fig.update_traces(textposition='top right')
fig.show()

In [ ]:
df_189_nod=df_189.groupby(['nod'])['nod'].count()
fig= px.bar(df_189_nod, text='value', labels={ 'index': 'hour', 'value':'count'}, title='Total count per day of week', width=1400,height=600)
fig.show()

In [ ]:
df_189_nod_mean=df_189[['dBA', 'nod']].groupby(['nod']).mean()
fig= px.line(df_189_nod_mean, text=df_189_nod_mean.index, title='Sound values aggregated by day of week', width=1500,height=600)
fig.update_traces(textposition='top right')
fig.show()

In [ ]:
df_189_monthly_count=df_189[['dBA', 'month']].groupby(['month']).count()
fig= px.bar(df_189_monthly_count, text='value', labels={ 'index': 'month', 'value':'count'}, title='Total count by month of year', width=1400,height=600)
fig.show()

In [ ]:
df_189_march= df_189.loc[df_189['month'] == 3]

In [ ]:
march10=df_189_march.loc[df_189_march['date'].isin(['2019-03-10'])]
march11=df_189_march.loc[df_189_march['date'].isin(['2019-03-11'])]
march12=df_189_march.loc[df_189_march['date'].isin(['2019-03-12'])]
march13=df_189_march.loc[df_189_march['date'].isin(['2019-03-13'])]
march14=df_189_march.loc[df_189_march['date'].isin(['2019-03-14'])]
march15=df_189_march.loc[df_189_march['date'].isin(['2019-03-15'])]

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots( rows=2, cols=3,horizontal_spacing=0.11)
    # subplot_titles=("Plot 1", "Plot 2", "Plot 3", "Plot 4", "Plot 5", "Plot7"))

fig.add_trace(go.Scatter(y=march10['dBA'], x=march10['time']),
              row=1, col=1)

fig.add_trace(go.Scatter(y=march11['dBA'], x=march11['time']),
              row=1, col=2)

fig.add_trace(go.Scatter(y=march12['dBA'], x=march12['time']),
              row=1, col=3)

fig.add_trace(go.Scatter(y=march13['dBA'], x=march13['time']),
              row=2, col=1)

fig.add_trace(go.Scatter(y=march14['dBA'], x=march14['time']),
              row=2, col=2)

fig.add_trace(go.Scatter(y=march15['dBA'], x=march15['time']),
              row=2, col=3)

fig.update_layout(height=700, width=1100)

fig.update_layout(
    plot_bgcolor='white'
)
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey',
    tickmode = 'array'
)
fig.update_yaxes(
    title_text="Noise Value (dBA)",
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
# fig.add_annotation(
#     text='(a), ', 
#     xref='paper', 
#     x=0.1, 
#     yref='paper', 
#     y=0.48, 
#     showarrow=False
# )
fig.update_layout(showlegend=False)
fig.show()

pio.write_image(fig, 'M_Subplots_Anom_Diff_Y_Axis.png',  scale=2)

In [ ]:
march17=df_189_march.loc[df_189_march['date'].isin(['2019-03-25'])]
march18=df_189_march.loc[df_189_march['date'].isin(['2019-03-26'])]
march19=df_189_march.loc[df_189_march['date'].isin(['2019-03-27'])]
march20=df_189_march.loc[df_189_march['date'].isin(['2019-03-28'])]
march21=df_189_march.loc[df_189_march['date'].isin(['2019-03-29'])]
march22=df_189_march.loc[df_189_march['date'].isin(['2019-03-30'])]

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots( rows=2, cols=3,horizontal_spacing=0.11)
    # subplot_titles=("Plot 1", "Plot 2", "Plot 3", "Plot 4", "Plot 5", "Plot7"))

fig.add_trace(go.Scatter(y=march17['dBA'], x=march17['time']),
              row=1, col=1)

fig.add_trace(go.Scatter(y=march18['dBA'], x=march18['time']),
              row=1, col=2)

fig.add_trace(go.Scatter(y=march19['dBA'], x=march19['time']),
              row=1, col=3)

fig.add_trace(go.Scatter(y=march20['dBA'], x=march20['time']),
              row=2, col=1)

fig.add_trace(go.Scatter(y=march21['dBA'], x=march21['time']),
              row=2, col=2)

fig.add_trace(go.Scatter(y=march22['dBA'], x=march22['time']),
              row=2, col=3)

fig.update_layout(height=700, width=1100)

fig.update_layout(
    plot_bgcolor='white'
)
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    title_text="Noise Value (dBA)",
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.update_layout(showlegend=False)
fig.show()

pio.write_image(fig, '3.png',  scale=2)

In [ ]:
df_189_march.loc[~df_189_march['date'].isin(['2019-03-10', '2019-03-11', '2019-03-12', '2019-03-13' ,'2019-03-14' ])].describe()

In [ ]:
df_189_march.loc[df_189_march['date'].isin(['2019-03-10', '2019-03-11', '2019-03-12', '2019-03-13' ,'2019-03-14' ])].describe()

In [ ]:
latex= pd.DataFrame(df_189_march.loc[df_189_march['date'].isin(['2019-03-10', '2019-03-11', '2019-03-12', '2019-03-13' ,'2019-03-14' ])]['dBA'].describe())
latex['anom']= df_189_march.loc[~df_189_march['date'].isin(['2019-03-10', '2019-03-11', '2019-03-12', '2019-03-13' ,'2019-03-14' ])]['dBA'].describe()

latex.to_latex()

In [ ]:
import plotly.io as pio
# import kaleido
fig= px.line(df_189_march, x='time', y='dBA', width=1200,height=600, labels={'dBA':'Noise Value (dBA)', 'time': 'Date'})
# fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    plot_bgcolor='white'
)
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    title_text="Noise Value (dBA)",
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.show()
pio.write_image(fig, '1.png',  scale=2)

In [ ]:
df_189.describe()

In [ ]:
print (df_189.loc[df_189['date'] < '2018-01-10'].count() +  df_189.loc[df_189['date'] > '2018-10-10'].count() + df_189.loc[(df_189['date'] >= '2018-01-10') & (df_189['date'] <= '2018-10-10')].count() )

**Data preparation for LSTM**

In [ ]:
df_189['month'].unique()

In [ ]:
# test = df_189.loc[(df_189['month'].isin ([3,9,10])) | df_189['date'].isin (['2019-11-11', '2019-11-30', '2019-12-10', '2019-12-14']) ]
test = df_189.loc[(df_189['month'].isin ([ 3 ])) ]
test.to_csv('/content/test.csv', index=False)

In [ ]:
# train= df_189.loc[~df_189.index.isin (test.index)]
train= df_189.loc[(df_189['month'].isin ([ 4,5,6,]))]
train.to_csv('/content/train.csv', index=False)
train.head()

In [ ]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
train.head()

In [ ]:
train=train.sort_values('time')
fig= px.line(train, x='time', y='dBA',  title='Lineplot of sound values')
fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
test=test.sort_values('time')
fig= px.line(test, x='time', y='dBA',  title='Lineplot of sound values')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
test.loc[((test['dBA'].between(28.5,29.4)) & (test['date'].isin(['2019-03-10', '2019-03-11', '2019-03-12', '2019-03-13', '2019-03-14']))) , 'Class']= 'Anomaly'


In [ ]:
test.loc[~((test['dBA'].between(28.5,29.4)) & (test['date'].isin(['2019-03-10', '2019-03-11', '2019-03-12', '2019-03-13', '2019-03-14']))) , 'Class']= 'Normal'

In [ ]:
test.groupby('Class')['dBA'].count()

In [ ]:
scaler= MinMaxScaler()
# scaler= MaxAbsScaler()
scaler = scaler.fit(train[['dBA']])

In [ ]:
train['dBA'] = scaler.transform(train[['dBA']])
test['dBA'] = scaler.transform(test[['dBA']])

In [ ]:
test['dBA'].min(), train['dBA'].min() , test['dBA'].max(), train['dBA'].max()

(-0.015761821366024498, 0.0, 1.071803852889667, 1.0)

In [ ]:
seq_size = 1
def to_sequences(x, y, seq_size=1):
    x_values = []
    y_values = []

    for i in range(len(x)-seq_size):
        #print(i)
        x_values.append(x.iloc[i:(i+seq_size)].values)
        y_values.append(y.iloc[i+seq_size])
        
    return np.array(x_values), np.array(y_values)

trainX, trainY = to_sequences(train[['dBA']], train['dBA'], seq_size)
testX, testY = to_sequences(test[['dBA']], test['dBA'], seq_size)
# testxX, testxY = to_sequences(testx[['dBA']], testx['dBA'], seq_size)

**LSTM Model**

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = Sequential()
model.add(LSTM(32, activation='swish', input_shape=(trainX.shape[1], trainX.shape[2]),return_sequences=False, unroll=True))
# model.add(LSTM(8, activation='swish', return_sequences=False, unroll=True))
# model.add(Dropout(rate=0.15))
model.add(RepeatVector(trainX.shape[1]))
# model.add(LSTM(8, activation='swish', return_sequences=True, unroll=True))
model.add(LSTM(32, activation='swish', return_sequences=True, unroll=True))
# model.add(Dropout(rate=0.15))
model.add(TimeDistributed(Dense(trainX.shape[2])))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="mae")
model.summary()


In [ ]:
history= model.fit(trainX, trainY, epochs=15, batch_size=32, validation_split=0.25 ,verbose=1, shuffle=False)
model.summary()

In [ ]:
loss= px.line(history.history, width=750, height=500, title= 'Training Loss Curve')
loss.show() 

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
# plt.yticks(np.arange(0.00, 0.50, 0.01))
plt.legend()

In [ ]:
model.evaluate(testX, testY)

1260/1260 [==============================] - 3s 2ms/step - loss: 0.0651


0.06506885588169098

In [ ]:
trainPredict = model.predict(trainX)

4050/4050 [==============================] - 8s 2ms/step


In [ ]:
trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
max_trainMAE = np.max(trainMAE)
hist= px.histogram(trainMAE, nbins=30, width=750,height=500, title='Train MAE Distribution')
hist.show()

In [ ]:
testPredict = model.predict(testX)
print(testPredict.shape)
print(testX.shape)

1260/1260 [==============================] - 2s 2ms/step
(40317, 1, 1)
(40317, 1, 1)


In [ ]:
testMAE = np.mean(np.abs(testPredict - testX), axis=1)
hist= px.histogram(testMAE, nbins=30, width=750,height=500, title='Test MAE Distribution')
hist.show()

In [ ]:
anomaly_df = pd.DataFrame(test[seq_size:])
anomaly_df['testMAE'] = testMAE
anomaly_df['max_trainMAE'] = max_trainMAE
anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE']
anomaly_df['dBA'] = test[seq_size:]['dBA']

In [ ]:
anomaly_df.loc[anomaly_df['anomaly'] == True]

In [ ]:
sns.lineplot(x=anomaly_df['date'], y=anomaly_df['testMAE'])
sns.lineplot(x=anomaly_df['date'], y=anomaly_df['max_trainMAE'])
anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]

In [ ]:
anomalies.loc[anomalies['Class']== 'Anomaly'].count()

In [ ]:
fig= px.line(x=anomaly_df['time'], y=anomaly_df['testMAE'],  title='Lineplot of sound values', labels={'x':'Time', 'y':'dBA'})
fig.add_scatter(x=anomaly_df['time'], y=anomaly_df['max_trainMAE'], mode='lines', name='Threshold')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
testX.reshape(1,-1).shape, testPredict.reshape(1,-1).shape

((1, 40317), (1, 40317))

In [ ]:
# x_test_orig= scaler.inverse_transform(testX.reshape(-1,1))
# x_pre_orig= scaler.inverse_transform(testPredict.reshape(-1,1))
# anomalies_df_values=scaler.inverse_transform(anomalies['dBA'].values.reshape(-1,1))

overlay_inverse=pd.DataFrame(scaler.inverse_transform(testX.reshape(-1,1)))
overlay_inverse['predict_original']= scaler.inverse_transform(testPredict.reshape(-1,1))
x1= overlay_inverse.index
y3= overlay_inverse[0]
y4= overlay_inverse['predict_original']

In [ ]:
import plotly.io as pio
overlay_inverse=pd.DataFrame(scaler.inverse_transform(testX.reshape(-1,1)))
overlay_inverse['predict_original']= scaler.inverse_transform(testPredict.reshape(-1,1))
x1= overlay_inverse.index
y3= overlay_inverse[0]
y4= overlay_inverse['predict_original']
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=x1, y=y3,
                    mode='markers',
                    name='Observed'))
fig.add_trace(go.Scatter(x=x1, y=y4,
                    mode='markers',
                    name='Predicted'))
fig.update_layout(
    plot_bgcolor='white',
    width=1281,
    height=785
    
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    title_text="Noise Value (dBA)",
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
pio.write_image(fig, 'obs_pred_189.png',  scale=2)
fig.show()

In [ ]:
overlay=pd.DataFrame(testX.reshape(-1,1))
overlay['A']= testPredict.reshape(-1,1)
x= overlay.index
y1= overlay[0]
y2= overlay['A']

In [ ]:
fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(x=x, y=y1,
                    mode='markers',
                    name='markers'))
fig.add_trace(go.Scatter(x=x, y=y2,
                    mode='markers',
                    name='anomaly'))
fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=anomaly_df['time'], y=b, name='Close price'))
# fig.add_trace(go.Scatter(x=anomalies['time'], y=b2, mode='markers', name='Anomaly'))
# fig.update_layout(showlegend=True, title='Detected anomalies')
# fig.show()

fig= px.line(x=anomaly_df['time'], y=anomaly_df['dBA'],  title='Lineplot of sound values', height=600)
fig.add_scatter(x=anomalies['time'], y=anomalies['dBA'], mode='markers', name='Detected Anomalies')
# fig.add_scatter(x=anomalies['time'], y=b2, mode= 'markers')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title={
        'text': "Lineplot of sound values",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
#INVERSE SCALER 
anomaly_df_values= scaler.inverse_transform(anomaly_df['dBA'].values.reshape(-1,1))
anomalies_df_values=scaler.inverse_transform(anomalies['dBA'].values.reshape(-1,1))
anomaly_df_values=pd.DataFrame(anomaly_df_values, columns=['dBA'])
anomalies_df_values=pd.DataFrame(anomalies_df_values, columns=['dBA'])


In [ ]:
anomalies_df_values

In [ ]:
#VISUALIZE ORIGINAL VALUES WITH ANOMALIES
fig= px.line(x=anomaly_df['time'], y=anomaly_df_values['dBA'], labels={'x':'Time', 'y':'dBA'})
fig.add_scatter(x=anomalies['time'], y=anomalies_df_values['dBA'], mode='markers', name= 'Detected Anomalies')
# fig.add_scatter(x=anomalies['time'], y=b2, mode= 'markers')
# fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
import plotly.io as pio
overlay_inverse=pd.DataFrame(scaler.inverse_transform(testX.reshape(-1,1)))
overlay_inverse['predict_original']= scaler.inverse_transform(testPredict.reshape(-1,1))
x1= overlay_inverse.index
y3= overlay_inverse[0]
y4= overlay_inverse['predict_original']
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=anomaly_df['time'], y=anomaly_df_values['dBA'],
                    mode='lines',
                    name='Observed'))
fig.add_trace(go.Scatter(x=anomalies['time'], y=anomalies_df_values['dBA'], mode='markers', name= 'Detected Anomalies',))

fig.update_layout(
    plot_bgcolor='white',
    width=1281,
    height=785
    
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))

fig.update_xaxes(
    title_text="Date",
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    title_text="Noise Value (dBA)",
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
pio.write_image(fig, 'detected_anomalies_189.png',  scale=2)
fig.show()

In [ ]:
anomalies.loc[(anomalies['anomaly'] == True) & (anomalies['Class'] == 'Normal')]

In [ ]:
 anomalies.loc[(anomalies['Class'] == 'Normal') & (anomalies['anomaly']== True)].count()

In [ ]:
 anomalies.loc[(anomalies['Class'] == 'Normal') & (anomalies['anomaly']== False)].count()

In [ ]:
 anomalies.loc[(anomalies['Class'] == 'Anomaly') & (anomalies['anomaly']== True)].count()

In [ ]:
true_negatives= anomalies['Class'].loc[anomalies['Class']== 'Anomaly'].count()
false_negatives= anomalies['Class'].loc[anomalies['Class']== 'Normal'].count()
true_positives= test['Class'].loc[test['Class']== 'Normal'].count() - false_negatives
false_positives= test['Class'].loc[test['Class']== 'Anomaly'].count() - true_negatives 
true_negatives, false_negatives, true_positives, false_positives

(5655, 120, 34397, 146)

In [ ]:
true_positives= anomalies['Class'].loc[anomalies['Class']== 'Anomaly'].count()
false_positives= anomalies['Class'].loc[anomalies['Class']== 'Normal'].count()
true_negatives= test['Class'].loc[test['Class']== 'Normal'].count() - false_positives
false_negatives= test['Class'].loc[test['Class']== 'Anomaly'].count() - true_positives 
true_positives, false_positives, true_negatives, false_negatives

(5655, 120, 34397, 146)

In [ ]:
con_matrix=pd.DataFrame.from_records(
    [["TensorFlow", true_positives, false_positives],
     ["TensorFlow Lite", false_negatives, true_negatives]],
     columns = ["Model", "Anomaly", "Normal"], index="Model")
con_matrix

,Size,Size_Comparison
Model,,
TensorFlow,5655,120
TensorFlow Lite,146,34397


In [ ]:
test.count()
normal=test['Class'].loc[test['Class']== 'Normal'].count()
abnormal=test['Class'].loc[test['Class']== 'Anomaly'].count()


In [ ]:
import plotly.io as pio
z = [[true_positives, false_positives],
     [false_negatives, true_negatives]]

x = ['Anomaly', 'Normal']
y = ['Anomaly', 'Normal']

z_text =  [[true_positives, false_positives],
     [false_negatives, true_negatives]]
fig = px.imshow(z, x=x, y=y, color_continuous_scale='blues', aspect="auto",  labels = {'x':'<b>Predicted Labels</b>', 'y': '<b>True Labels</b>'}, width=770,height=500)
fig.update_traces(text=z_text, texttemplate="%{text}")
fig.update_xaxes(side="top")
pio.write_image(fig, 'con_mat_189.png',  scale=2)
fig.show()

In [ ]:
precision = true_positives/(true_positives + false_positives)
recall = true_positives/(true_positives + false_negatives)
f1_score = 2 * ((precision * recall)/(precision + recall))
accuracy = (true_positives + true_negatives)/(true_positives + true_negatives + false_positives + false_negatives)

precision, recall, f1_score, accuracy

(0.9792207792207792, 0.974831925530081, 0.9770214236351071, 0.9934024505183789)

In [ ]:
precision = 1888/(1888 + 212)
recall = 1888/(1888 + 0)
f1_score = 2 * ((precision * recall)/(precision + recall))
accuracy = (1888 + 40697)/(1888  + 40697 + 212 + 0)

precision, recall, f1_score, accuracy

(0.8990476190476191, 1.0, 0.9468405215646941, 0.9950463817557306)

In [ ]:
precision = 1888/(1888 + 212)
recall = 1888/(1888 + 0)
f1_score = 2 * ((precision * recall)/(precision + recall))
accuracy = (1888 + 40697)/(1888  + 40697 + 212 + 0)

precision, recall, f1_score, accuracy

In [ ]:
model.save(MODEL_TF)

**Tensorflow Lite**

In [ ]:
# MODEL CONVERSION TFLITE
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

61868

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=model_no_quant_tflite)

**Tensorflow Lite Quantized**

In [ ]:
testX32 =testX.astype(np.float32)

In [ ]:
testX32[:300].reshape(1,-1)
testX32
mcu_test= pd.DataFrame(testX32.reshape(-1,1))
mcu_test= mcu_test[:300]
mcu_test.to_csv('mcu_test_189.csv')

In [ ]:
def representative_dataset_gen(num_samples=50000):
  for data in testX32[:num_samples]:
    yield [data.reshape(1,1,1)]

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert

In [ ]:
open(MODEL_TFLITE, "wb").write(tflite_quant_model)

26792

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_quant_model)

In [ ]:
# https://www.tensorflow.org/lite/performance/post_training_integer_quant#convert_using_integer-only_quantization
TEST_CASES=40317
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]


prediction= []
for i in range(TEST_CASES):
  if input_details['dtype'] == np.int8:
    input_scale, input_zero_point = input_details["quantization"]
    output_scale, output_zero_point = output_details["quantization"]
    test_int8 = testX32 / input_scale + input_zero_point
    test_int8= test_int8.astype(np.int8)
    
  interpreter.set_tensor(input_details["index"], test_int8[i:i+1, :, :])
  # interpreter.set_tensor(input_details["index"], test_image)
  interpreter.invoke()
  output = interpreter.get_tensor(output_details["index"])[0]
  y= (output.astype(float) - output_zero_point) * output_scale

  prediction.append(y)



In [ ]:
predictions = np.array(prediction)

In [ ]:
tflite_predictions = pd.DataFrame(predictions.reshape(-1,1))
tflite_predictions.columns = ['y_value']
tflite_predictions

In [ ]:
#Mean Squared Error 
testPre_Inv=scaler.inverse_transform(testPredict.reshape(1, -1))
testPreMCU_Inv= scaler.inverse_transform(predictions.reshape(1, -1))
testX_Inv=scaler.inverse_transform(testX.reshape(1, -1))
testY_Inv= scaler.inverse_transform(testY.reshape(1, -1))

trainX_Inv=scaler.inverse_transform(trainX.reshape(1, -1))
trainY_Inv= scaler.inverse_transform(trainY.reshape(1, -1))

from sklearn.metrics import mean_squared_error

mse_original_train= mean_squared_error(trainX_Inv, trainY_Inv, squared=False)
mse_original= mean_squared_error(testPre_Inv, testY_Inv, squared=False)
mse_mcu= mean_squared_error(testPreMCU_Inv, testY_Inv, squared=False)

# mse2 = mean_squared_error(np_arr.reshape(1,-1), testY.reshape(1,-1))
print("Tensorflow RMSE for testing data is :" , mse_original , "\n" + "Tensorflow Lite RMSE for testing data is: ", mse_mcu, "\n" + "Tensorflow Lite RMSE for training data is: ", mse_original_train)

In [ ]:
#TENSORFLOW ORIGINAL AND TENSORFLOW LITE OVERLAY
import kaleido
import plotly.io as pio
test_Pre_Overlay= pd.DataFrame(testPre_Inv.reshape(-1,1))
test_Pre_Overlay.columns = ['y_value']
test_Pre_MCU_Overlay= pd.DataFrame(testPreMCU_Inv.reshape(-1,1))
test_Pre_MCU_Overlay.columns = ['y_value']

fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(y=test_Pre_Overlay['y_value'][14000:14101],
                    mode='lines',
                    name='TensorFlow Original'))
fig.add_trace(go.Scatter(y=test_Pre_MCU_Overlay['y_value'][14000:14101],
                    mode='lines',
                    name='TensorFlow Lite'))
fig.update_layout(
    plot_bgcolor='white',
    width=1281,
    height=785  
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    title_text="Noise Value (dBA)",
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
pio.write_image(fig, 'tf_tflite_diff_f14k.png',  scale=2)
fig.show()

In [ ]:
testY_Inv.mean()

47.64127787285761

In [ ]:
(3.19/47.6412778728576) * 100


6.695874129391102

In [ ]:
testY[1], testX[1], test_int8[1].astype(float), test_int8.dtype

**C CODE**

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [ ]:
!cat {MODEL_TFLITE_MICRO}

**COMPARE MODEL SIZE**

In [ ]:
# Calculate size
size_tf = os.path.getsize('/content/models/model/saved_model.pb')
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)

In [ ]:
# Compare size
Size=pd.DataFrame.from_records(
    [["TensorFlow", size_tf, ""],
     ["TensorFlow Lite", size_no_quant_tflite, f"(reduced by {size_tf - size_no_quant_tflite} bytes)"],
     ["TensorFlow Lite Quantized", size_tflite, f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
     columns = ["Model", "Size", "Size_Comparison"], index="Model")

In [ ]:
Size

,Size,Size_Comparison
Model,,
TensorFlow,220824,
TensorFlow Lite,60992,(reduced by 159832 bytes)
TensorFlow Lite Quantized,25896,(reduced by 35096 bytes)


In [ ]:
fig = px.bar(Size, x=Size.index, y='Size', text_auto=True, width=750,height=500, labels={'Size': 'Size (kB)', 'Model': 'Models'})
fig.show()